# An advanced look at LLM Agents
## Notebook 2: Building an agent using the GPT API chat completion endpoint
---

Previously, we relied on LangChain to to build the agent. However, this is not necessary, since an agent is nothing more than a fancy while loop. 

**The goal** 

With this notebook you will see what an agent is under the hood, and you can build it based on the LLM output 

**Contents:**

1. [Exercise 1: New tool format](#1)
2. [Exercise 2: String vs function call response](#2)
3. [Exercise 3: Understanding the agent while-loop](#3)

### Setup Google Colab (if not run locally)
To set up Google Colab, follow the steps below. If you would rather run the notebooks locally, visit the `README`

1. Open [Google colab](https://colab.research.google.com/) > Open notebook > GitHub 
2. Paste the [repository link](https://github.com/mkmbader/pydata_workshop_September2024) 
3. Click on “2_workshop_agent_with_chat_completion_endpoint.ipynb''

In [ ]:
# RUN THIS IF YOU USE GOOGLE COLAB. 
# Otherwise you can COMMENT IT OUT and set up the repository LOCALLY (see instructions in README)

!mkdir helper_functions/
!mkdir images/
!curl https://raw.githubusercontent.com/mkmbader/pydata_workshop_September2024/master/requirements.txt > requirements.txt
!curl https://raw.githubusercontent.com/mkmbader/pydata_workshop_September2024/master/helper_functions/helper_functions.py > helper_functions/helper_functions.py
!curl https://raw.githubusercontent.com/mkmbader/pydata_workshop_September2024/master/helper_functions/keys.py > helper_functions/keys.py
!curl https://raw.githubusercontent.com/mkmbader/pydata_workshop_September2024/master/helper_functions/tools.py > helper_functions/tools.py

!pip install -r requirements.txt

🌟 So ... let us begin! 

----

In [ ]:
import sys
import os
current_dir = os.path.dirname(os.path.abspath('.'))
folder_b_path = os.path.join(current_dir, 'helper_functions')
sys.path.append(current_dir)

from helper_functions.tools import my_own_wiki_tool, weather_tool
from helper_functions.keys import client
import json

## The tools

When querying OpenAI's API, tools are called via **function calling** (see the API documentation on function calling [here](https://platform.openai.com/docs/assistants/tools/function-calling/quickstart)). Functions have to be passed in JSON format, which we will explore below.


#### Exercise 1: New tool format <a id='1'></a>

First, we'll look at an example of how to format the `my_own_wiki_tool` into a function json, then you'll format the `weather_tool` into a function json the same way.

**TASK:**  
Compile the next two cells and have a look at the example. Do you understand the output?

In [ ]:
# Example: print attributes of my_own_wiki_tool

print('name: ', my_own_wiki_tool.name)
print('description: ', my_own_wiki_tool.description)
print('arguments: ', my_own_wiki_tool.args)

In [ ]:
# Example: json format of the my_own_wiki_tool

my_own_wiki_tool_json = {
    "type": "function",
    "function": {
        "name": my_own_wiki_tool.name,
        "description": my_own_wiki_tool.description,
        "parameters": {
            "type": "object",
            # below all individual function parameters need to be listed
            "properties": {
                'query':{
                    'description':'Input search query.',
                    'type':'string',
                }
            },
            "required": ['query']
        },
    },
}


**TASK:**  
Using the example above for guidance, create the json format for the weather tool. If you have trouble filling in the attributes, first print them from the tool.

In [ ]:
weather_tool_json = {
    "type": "function",
    "function": {
        "name": # <TODO: your code here>,
        "description": # <TODO: your code here>,
        "parameters": {
            "type": "object",
            # below all individual function parameters need to be listed
            "properties": {
                # <TODO: your code here>
                }
            },
            "required": # <TODO: your code here>
        },
    },
}

Next, the jsonized functions are combined in a list, while the actual tools are stored in a dictionary. After this we have prepared the callable functions and are ready to interact with the chat completions endpoint.

**TASK:**  
Add the `weather_tool` by completing the code below

In [ ]:
# callable functions 
callable_functions = [my_own_wiki_tool_json, #<TODO: your code here>]

# Store executable functions with their name in dictionary
available_functions = {
    my_own_wiki_tool.name :my_own_wiki_tool, 
    # <TODO: your code here>
    }

---

## The LLM response to input questions

The LLM can respond with two types of answers to input queries:
* a **string** that answers the question, 
* a **function-call** object, which contains information on which function to call with which arguments.

The second one can be used to execute function calls.

#### Exercise 2: String vs function call response <a id='2'></a>

**TASK:**
Compile both questions and compare the answers. Do you understand the difference?


In [ ]:
system_prompt = "You are a friendly, helpful assistant. Your goal is to answer the questions in a concise, but conversational manner."

questions = ["what is the meaning of life?","What temperature is it in Paris?"]

for question in questions:
  messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": question}
    ]

  
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    tools = callable_functions,
    messages=messages,
    )

  print(f"Question: {question}")
  print(f"Answer: {response.choices[0].message.content}")
  print(f"Function call: {response.choices[0].message.tool_calls}\n")

**TASK:**  
For the function call object above, extract the **name** and the **arguments** of the function call and print them.

In [ ]:
# <TODO: Fill in your code here>

---

## The Agent - a fancy while loop

While the LLM requests function calls we 
* **extract** the **name and arguments** to be called from the initial LLM response,
* **execute** the **function calls**,
* **store** the **output of the function** in the messages object,
* invoke the LLM again, until no function call are requested.

For more details, you can also check out this [OpenAI function calling guide](https://platform.openai.com/docs/guides/function-calling).

#### Exercise 3: Understanding the agent while-loop <a id='3'></a>

**TASK:**
Complete the code below. Then compile the question and investigate the output. Do you understand what you see?

In [ ]:
question = "which city is bigger: Paris or Munich?"

messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": question}
    ]

response = client.chat.completions.create(
  model="gpt-4o-mini",
  tools = callable_functions,
  messages=messages, 
  tool_choice='required',
  )

print('==== Initial LLM response ====')
print(f"Answer: {response.choices[0].message.content}")
print(f"Function call: {response.choices[0].message.tool_calls}\n")

# while the response requests function calls
# TASK: check above how to extract the function call object from the chat completion object. Fill it in below.
while # <TODO: fill in your code here>:
    
  # store response message with all function calls
  response_message = response.choices[0].message
  messages.append(response_message)

  # execute each tool individually
  for tool_call in response.choices[0].message.tool_calls:
    print('==== Function call ====')

    # TASK: EXTRACT FUNCTION NAME AND ARGUMENTS BELOW
    # function name and arguments
    function_name = #<TODO: fill in your code here>
    function_args = json.loads(#<TODO: fill in your code here>)
    print(f'Calling function "{function_name}" with arguments {function_args}.')

    # execute function call 
    function_response = available_functions[function_name].invoke(function_args)
    print(f'Function call response:\n{function_response}\n')

    # append function response to messages
    messages.append({
        "tool_call_id":tool_call.id, 
        "role": "tool", 
        "name": function_name, 
        "content": function_response
    })
    
  # get a new response from LLM
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    tools = callable_functions,
    messages=messages, 
  )

  print('==== Intermediate LLM response ====')
  print(f"Answer: {response.choices[0].message.content} ")
  print(f"Function call: {response.choices[0].message.tool_calls }\n")

print('==== Final LLM response ====')
print("Question: ", question)
print(f"Answer: {response.choices[0].message.content}")
print(f"Function call: {response.choices[0].message.tool_calls}\n")

🌟 Congratulations - you've finished the workshop